## Instalar Librerías e Importar el Dataset

In [ ]:
%pip install ucimlrepo tensorflow aif360 'aif360[Reductions]' 'aif360[inFairness]'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'aif360[Reductions]'"

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from ucimlrepo import fetch_ucirepo
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [4]:
# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

## Preparar el Dataset

In [5]:
def clean_labels(label: str):
    return label.replace(".", "").replace(" ", "_")

X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y)

y_df = y_df.map(clean_labels)

df = pd.concat([X_df, y_df], axis=1)
df.dropna(inplace=True)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
df.shape

(47621, 15)

In [7]:
X_df_clean = df.drop(columns=['income'])
y_df_clean = df['income']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_df_clean, y_df_clean, test_size=0.3, stratify=y_df_clean, random_state=42)

In [9]:
X_train = pd.DataFrame(X_train, columns=X_df_clean.columns)
X_test = pd.DataFrame(X_test, columns=X_df_clean.columns)

y_train = pd.DataFrame(y_train, columns=['income'])
y_test = pd.DataFrame(y_test, columns=['income'])

In [10]:
# Sex map
mapping_sex = {"Female": 0, "Male": 1}
# Income map
mapping_income = {"<=50K": 0, ">50K": 1}

def map_col(
    df: pd.DataFrame,
    col: str,
    mapping: dict,
) -> pd.DataFrame:
    df[col] = df[col].map(mapping)
    return df

In [11]:
# Preprocessing train data
X_train['age'] = X_train['age'].apply(lambda x: 1 if x < 60 else 0)
X_train = map_col(X_train, "sex", mapping_sex)

y_train = map_col(y_train, "income", mapping_income)

# Preprocessing test data
X_test['age'] = X_test['age'].apply(lambda x: 1 if x < 60 else 0)
X_test = map_col(X_test, "sex", mapping_sex)

y_test = map_col(y_test, "income", mapping_income)

In [12]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
26818,1,Private,202033,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,1,0,0,40,United-States
28252,1,Private,340599,11th,7,Separated,Other-service,Unmarried,Black,0,0,0,40,United-States
9258,1,State-gov,177035,11th,7,Divorced,Other-service,Unmarried,White,0,0,0,40,United-States
41064,1,Self-emp-not-inc,30012,Some-college,10,Married-civ-spouse,Sales,Husband,White,1,0,0,50,United-States
30461,1,Private,185216,Some-college,10,Never-married,Adm-clerical,Own-child,White,1,0,0,40,United-States


In [13]:
y_train.head()

,income
26818,0
28252,0
9258,0
41064,1
30461,0


In [14]:
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

standard_scaler = MinMaxScaler()
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

# Aplicamos directamente a los datos
X_numeric_train_scaled = standard_scaler.fit_transform(X_train[numeric_cols])
X_numeric_test_scaled = standard_scaler.transform(X_test[numeric_cols])

# Aplicamos one hot encoding
X_categorical_train = one_hot_encoder.fit_transform(X_train[categorical_cols])
X_categorical_test = one_hot_encoder.transform(X_test[categorical_cols])

X_train_processed = np.concatenate([X_numeric_train_scaled, X_categorical_train.toarray()], axis=1)
X_test_processed = np.concatenate([X_numeric_test_scaled, X_categorical_test.toarray()], axis=1)
X_train_processed.shape, X_test_processed.shape

((33334, 107), (14287, 107))

In [15]:
X_train_df = pd.DataFrame(X_train_processed, columns=numeric_cols.tolist() + one_hot_encoder.get_feature_names_out(categorical_cols).tolist())
X_test_df = pd.DataFrame(X_test_processed, columns=numeric_cols.tolist() + one_hot_encoder.get_feature_names_out(categorical_cols).tolist())

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [16]:
X_train_df.head()

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,1.0,0.127659,0.8,1.0,0.0,0.0,0.397959,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.221481,0.4,0.0,0.0,0.0,0.397959,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.110733,0.4,0.0,0.0,0.0,0.397959,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.011186,0.6,1.0,0.0,0.0,0.500000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.116273,0.6,1.0,0.0,0.0,0.397959,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Entrenamiento de Modelos
### Logistic Regression Model

In [17]:
logistic_regression = LogisticRegression(max_iter=1000)

logistic_regression.fit(X_train_df, y_train)
logistic_pred = logistic_regression.predict(X_test_df)
acc_lr = accuracy_score(y_test, logistic_pred)
print(f'Accuracy for Logistic Regression Model: {acc_lr:.2f}')

Accuracy for Logistic Regression Model: 0.85


### Random Forest Classifier Model

In [18]:
random_forest = RandomForestClassifier()

random_forest.fit(X_train_df, y_train)
randomF_pred = random_forest.predict(X_test_df)
acc_rf = accuracy_score(y_test, randomF_pred)
print(f'Accuracy for Random Forest Classifier Model: {acc_rf:.2f}')

Accuracy for Random Forest Classifier Model: 0.83


### K Neighbors Classifier Model

In [19]:
knn = KNeighborsClassifier()

knn.fit(X_train_df, y_train)
knn_pred = knn.predict(X_test_df)
acc_kn = accuracy_score(y_test, knn_pred)
print(f'Accuracy for K Neighbors Classifier Model: {acc_kn:.2f}')

Accuracy for K Neighbors Classifier Model: 0.82


# Métricas de Equidad
#### Modelo escogido por mejor rendimiento: **Logistic Regression Model**
#### Atributos sensibles a estudiar: **age** y **sex**

## Independencia (Demographic Parity)

In [20]:
best_pred = logistic_pred

In [21]:
from aif360.datasets import BinaryLabelDataset

df_test_aif = BinaryLabelDataset(
    df=pd.concat([X_test_df, pd.DataFrame(y_test, columns=['income'])], axis=1),
    label_names=['income'],
    protected_attribute_names=['age', 'sex'],
)

df_test_pred_aif = df_test_aif.copy(deepcopy=True)
df_test_pred_aif.labels = best_pred.reshape(-1, 1)

df_train_aif = BinaryLabelDataset(
    df=pd.concat([X_train_df, pd.DataFrame(y_train, columns=['income'])], axis=1),
    label_names=['income'],
    protected_attribute_names=['age', 'sex'],
)

In [22]:
from aif360.metrics import ClassificationMetric


def get_demographic_parity_metrics(
    test_aif: BinaryLabelDataset,
    test_pred_aif: BinaryLabelDataset,
):
    # Para 'age'
    metric_age = ClassificationMetric(
        test_aif,
        test_pred_aif,
        privileged_groups=[{"age": 1}],
        unprivileged_groups=[{"age": 0}]
    )

    # Para 'sex'
    metric_sex = ClassificationMetric(
        test_aif,
        test_pred_aif,
        privileged_groups=[{"sex": 1}],
        unprivileged_groups=[{"sex": 0}]
    )

    print("Disparate Impact for age:", metric_age.disparate_impact())
    print("Disparate Impact for sex:", metric_sex.disparate_impact())
    
    return metric_age, metric_sex

In [23]:
get_demographic_parity_metrics(df_test_aif, df_test_pred_aif)

Disparate Impact for age: 1.050856421312152
Disparate Impact for sex: 0.2816774955165621


(<aif360.metrics.classification_metric.ClassificationMetric at 0x1653b632030>,
 <aif360.metrics.classification_metric.ClassificationMetric at 0x1653b5e6db0>)

## Separación (Equalized Odds)

In [24]:
def map_col(df, col, mapping) -> pd.DataFrame:
    df[col] = df[col].map(mapping)
    return df

def calculate_tpr_fpr_sex(data, subgroup):
    group_data = data[data['sex'] == subgroup]
    true_positive = np.sum((group_data['y_pred'] == 1) & (group_data['y_true'] == 1))
    false_positive = np.sum((group_data['y_pred'] == 1) & (group_data['y_true'] == 0))
    total_positive = np.sum(group_data['y_true'] == 1)
    total_negative = np.sum(group_data['y_true'] == 0)

    tpr = true_positive / total_positive if total_positive > 0 else 0
    fpr = false_positive / total_negative if total_negative > 0 else 0

    return tpr, fpr

def calculate_tpr_fpr_age(data, subgroup):
    group_data = data[data['age'] == subgroup]
    true_positive = np.sum((group_data['y_pred'] == 1) & (group_data['y_true'] == 1))
    false_positive = np.sum((group_data['y_pred'] == 1) & (group_data['y_true'] == 0))
    total_positive = np.sum(group_data['y_true'] == 1)
    total_negative = np.sum(group_data['y_true'] == 0)

    tpr = true_positive / total_positive if total_positive > 0 else 0
    fpr = false_positive / total_negative if total_negative > 0 else 0

    return tpr, fpr

In [25]:
def prepare_df_to_separacion(X_test_dataframe, y_test, y_pred):
    return pd.concat([
        X_test_dataframe,
        pd.DataFrame(y_test, columns=['y_true']),
        pd.DataFrame(y_pred, columns=['y_pred'])
    ], axis=1)

### Equalized Odds for **age** and **sex** atribute

Map: [**0: Older Adults**, **1: Young Adults**]

Map: [**0: Female**, **1: Male**]

In [26]:
def get_equalized_odds_metrics(
    X_df: pd.DataFrame,
    y_true: np.ndarray,
    y_pred: np.ndarray,
):
    separacion_df = prepare_df_to_separacion(X_df, y_true, y_pred)
    for age in [0, 1]:
        tpr, fpr = calculate_tpr_fpr_age(separacion_df, age)
        age_label = 'Older Adults' if age == 0 else 'Young Adults'
        print(f"Age Group: {age_label}")
        print(f"  True Positive Rate (TPR): {tpr:.2f}")
        print(f"  False Positive Rate (FPR): {fpr:.2f}")
        
    for sex in [0, 1]:
        tpr, fpr = calculate_tpr_fpr_sex(separacion_df, sex)
        sex_label = 'Female' if sex == 0 else 'Male'
        print(f"Sex Group: {sex_label}")
        print(f"  True Positive Rate (TPR): {tpr:.2f}")
        print(f"  False Positive Rate (FPR): {fpr:.2f}")

In [27]:
get_equalized_odds_metrics(X_test_df, y_test, best_pred)

Age Group: Older Adults
  True Positive Rate (TPR): 0.57
  False Positive Rate (FPR): 0.07
Age Group: Young Adults
  True Positive Rate (TPR): 0.59
  False Positive Rate (FPR): 0.07
Sex Group: Female
  True Positive Rate (TPR): 0.45
  False Positive Rate (FPR): 0.02
Sex Group: Male
  True Positive Rate (TPR): 0.61
  False Positive Rate (FPR): 0.10


## Suficiencia (Predictive Parity)

In [28]:
from aif360.metrics import ClassificationMetric


def get_predictive_parity_metrics(
    test_aif: BinaryLabelDataset,
    test_pred_aif: BinaryLabelDataset,
):
    # Para 'age'
    metric_age = ClassificationMetric(
        test_aif,
        test_pred_aif,
        privileged_groups=[{"age": 1}],
        unprivileged_groups=[{"age": 0}]
    )
    
    # Para 'sex'
    metric_sex = ClassificationMetric(
        df_test_aif,
        df_test_pred_aif,
        privileged_groups=[{"sex": 1}],
        unprivileged_groups=[{"sex": 0}]
    )

    # Calcular el Valor Predictivo Positivo (PPV) para 'age'
    privileged_ppv_age = metric_age.positive_predictive_value(privileged=True)
    unprivileged_ppv_age = metric_age.positive_predictive_value(privileged=False)

    # Calcular el Valor Predictivo Positivo (PPV) para 'sex'
    privileged_ppv_sex = metric_sex.positive_predictive_value(privileged=True)
    unprivileged_ppv_sex = metric_sex.positive_predictive_value(privileged=False)

    # Calcular la diferencia de paridad predictiva
    predictive_parity_difference_sex = privileged_ppv_sex - unprivileged_ppv_sex
    predictive_parity_difference_age = privileged_ppv_age - unprivileged_ppv_age
    
    print(f"Privileged PPV age: {privileged_ppv_age}")
    print(f"Unprivileged PPV age: {unprivileged_ppv_age}")
    print(f"Predictive Parity Difference age: {predictive_parity_difference_age}")

    print(f"Privileged PPV sex: {privileged_ppv_sex}")
    print(f"Unprivileged PPV sex: {unprivileged_ppv_sex}")
    print(f"Predictive Parity Difference sex: {predictive_parity_difference_sex}")

In [29]:
get_predictive_parity_metrics(df_test_aif, df_test_pred_aif)

Privileged PPV age: 0.7272727272727273
Unprivileged PPV age: 0.7467248908296943
Predictive Parity Difference age: -0.019452163556966995
Privileged PPV sex: 0.7310683585755219
Unprivileged PPV sex: 0.7130177514792899
Predictive Parity Difference sex: 0.018050607096231963


# Mitigación de Sesgos
### Pre-procesamiento: **Reweighing**
### In-procesamiento:
### Post-procesamiento:

## Pre-procesamiento: **Reweighing**

In [30]:
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset
from typing import List

def reweighingPreprocessing(
    train_aif_df: BinaryLabelDataset,
    sensitive_features: List[str],
):
    """
    Aplica Reweighing a un conjunto de entrenamiento considerando múltiples atributos sensibles.
    """

    # Crear procesadores Reweighing para cada atributo sensible
    reweigh_processors = []
    for sensitive_feature in sensitive_features:
        reweigh_processors.append(
            Reweighing(
                unprivileged_groups=[{sensitive_feature: 0}],
                privileged_groups=[{sensitive_feature: 1}]
            )
        )
    
    # Aplicar Reweighing a los pesos del dataset
    actual_train_df = train_aif_df.copy()
    
    for reweigh_processor in reweigh_processors:
        reweigh_processor: Reweighing
        reweigh_processor.fit(actual_train_df)
        actual_train_df = reweigh_processor.transform(actual_train_df)
    
    return actual_train_df

# Aplicar Reweighing
reweighted_train = reweighingPreprocessing(
    df_train_aif,
    ['age', 'sex']
)

# Mostrar los nuevos pesos después del preprocesamiento
print("Nuevos pesos tras Reweighing:")
print(reweighted_train.instance_weights[:10])  # Muestra los primeros 10 ejemplos

Nuevos pesos tras Reweighing:
[1.09493593 0.8507266  0.8507266  0.78650541 1.09493593 1.09493593
 1.09493593 0.8507266  1.09493593 0.8507266 ]


In [31]:
logistic_model_pre = LogisticRegression(max_iter=5000, random_state=42)
logistic_model_pre.fit(
    reweighted_train.features, 
    reweighted_train.labels.ravel(), 
    sample_weight=reweighted_train.instance_weights
)
y_pred_logistic_pre = logistic_model_pre.predict(X_test_df)
acc_lr_pre = accuracy_score(y_test, y_pred_logistic_pre)
print(f'Accuracy for Logistic Regression Model with Pre-processing: {acc_lr_pre:.2f}')

Accuracy for Logistic Regression Model with Pre-processing: 0.84


c:\Users\lucas\python-envs\nb-env\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


## In-procesamiento: **Inserte Técnica Aquí**

In [32]:
from typing import List
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
import tensorflow as tf 

def adversarialDebiasingProcessing(
    train_aif_df: BinaryLabelDataset,
    test_aif_df: BinaryLabelDataset,
    sensitive_features: List[str],
    num_epochs: int = 50,
    batch_size: int = 128,
    adversary_loss_weight: float = 0.1,
    name_run_identifier: str = 'adversarial_debiasing',
):
    """
    Función para entrenar un modelo debiasado usando AdversarialDebiasing.

    Parámetros:
    - train_aif_df (BinaryLabelDataset): Datos de entrenamiento.
    - test_aif_df (BinaryLabelDataset): Datos de prueba.
    - sensitive_features (List[str]): Lista de características sensibles (e.g., ['age', 'sex']).
    - num_epochs (int): Número de épocas para entrenar.
    - batch_size (int): Tamaño de lote durante el entrenamiento.
    - adversary_loss_weight (float): Peso del adversario para reducir el sesgo.

    Retorno:
    - predicted_test_aif_df (BinaryLabelDataset): Predicciones debiasadas en el conjunto de prueba.
    """

    tf.compat.v1.disable_eager_execution()
    sess = tf.compat.v1.Session()
    debiased_models = []
    predicted_test_aif_df = test_aif_df.copy()

    for sensitive_feature in sensitive_features:
        # Configurar grupos privilegiados y no privilegiados
        privileged_groups = [{sensitive_feature: 1}]
        unprivileged_groups = [{sensitive_feature: 0}]
        
        # Crear y entrenar el modelo debiasado
        debiased_model = AdversarialDebiasing(
            privileged_groups=privileged_groups,
            unprivileged_groups=unprivileged_groups,
            scope_name=f'{name_run_identifier}_{sensitive_feature}',
            sess=sess,
            num_epochs=num_epochs,
            batch_size=batch_size,
            adversary_loss_weight=adversary_loss_weight
        )
        print(f"Entrenando modelo debiasado para {sensitive_feature}...")
        debiased_model.fit(train_aif_df)
        debiased_models.append(debiased_model)

        # Generar predicciones
        predicted_test_aif_df = debiased_model.predict(predicted_test_aif_df)

    sess.close()
    return predicted_test_aif_df


In [33]:
predicted_test_inprocessing = adversarialDebiasingProcessing(
    train_aif_df=df_train_aif,  # Datos de entrenamiento
    test_aif_df=df_test_aif,    # Datos de prueba
    sensitive_features=['age', 'sex'],  # Atributos sensibles
    num_epochs=50,  # Configuración del número de épocas
    batch_size=128,  # Tamaño de lote
    adversary_loss_weight=0.1,  # Peso del adversario
    name_run_identifier='test_1'
)

Entrenando modelo debiasado para age...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.693852; batch adversarial loss: 0.657653
epoch 0; iter: 200; batch classifier loss: 0.461706; batch adversarial loss: 0.582487
epoch 1; iter: 0; batch classifier loss: 0.439766; batch adversarial loss: 0.563201
epoch 1; iter: 200; batch classifier loss: 0.464464; batch adversarial loss: 0.539654
epoch 2; iter: 0; batch classifier loss: 0.353233; batch adversarial loss: 0.550909
epoch 2; iter: 200; batch classifier loss: 0.443546; batch adversarial loss: 0.473871
epoch 3; iter: 0; batch classifier loss: 0.454370; batch adversarial loss: 0.421975
epoch 3; iter: 200; batch classifier loss: 0.513054; batch adversarial loss: 0.461757
epoch 4; iter: 0; batch classifier loss: 0.553348; batch adversarial loss: 0.377774
epoch 4; iter: 200; batch classifier loss: 0.386324; batch adversarial loss: 0.418895
epoch 5; iter: 0; batch classifier loss: 0.428137; batch adversarial loss: 0.354470
epoch 5; iter: 200; batch classifier loss: 0.260522; batch adversa

## Post-procesamiento: **Equalized Odds Post-Processing**

In [34]:
from typing import List

#Código Post-Procesamiento
from aif360.algorithms.postprocessing import EqOddsPostprocessing

# Cramos una función que se encargue de reajustar las predicciones
def eqOddsPredictionProccesing(
    test_aif_df: BinaryLabelDataset,
    test_pred_aif_df: BinaryLabelDataset,
    sensitive_features: List[str],
):

    eq_odds_processers = []
    for sensitive_feature in sensitive_features:
        eq_odds_processers.append(
            EqOddsPostprocessing(
                unprivileged_groups=[{sensitive_feature: 0}],
                privileged_groups=[{sensitive_feature: 1}],
                seed=42
            )
        )
    
    actual_pred_aif_df = test_pred_aif_df.copy()
    
    for eq_odds_processer in eq_odds_processers:
        eq_odds_processer: EqOddsPostprocessing
        eq_odds_processer.fit(test_aif_df, actual_pred_aif_df)
        actual_pred_aif_df = eq_odds_processer.predict(actual_pred_aif_df)
        
    return actual_pred_aif_df

In [35]:
post_processed_preds = eqOddsPredictionProccesing(
    df_test_aif,
    df_test_pred_aif, 
    ['age', 'sex']
)

get_demographic_parity_metrics(
    df_test_aif,
    post_processed_preds
)

new_accuracy = accuracy_score(y_test, post_processed_preds.labels)
print(f"New Accuracy: {new_accuracy}")

Disparate Impact for age: 1.0650931315940388
Disparate Impact for sex: 0.6202328376320687
New Accuracy: 0.8115069643732065


# Medición de Mitigación de Sesgos

## Combinación 1: Pre-procesamiento + In-Procesamiento

In [36]:
## Código que combine las dos técnicas
reweighted_train = reweighingPreprocessing(
    df_train_aif,
    ['age', 'sex']
)

predicted_test_inprocessing_1 = adversarialDebiasingProcessing(
    train_aif_df=reweighted_train,  # Datos de entrenamiento
    test_aif_df=df_test_aif,    # Datos de prueba
    sensitive_features=['age', 'sex'],  # Atributos sensibles
    num_epochs=50,  # Configuración del número de épocas
    batch_size=128,  # Tamaño de lote
    adversary_loss_weight=0.1,  # Peso del adversario
    name_run_identifier='test_2'
)

Entrenando modelo debiasado para age...
epoch 0; iter: 0; batch classifier loss: 0.654990; batch adversarial loss: 0.544777
epoch 0; iter: 200; batch classifier loss: 0.380507; batch adversarial loss: 0.603652
epoch 1; iter: 0; batch classifier loss: 0.395995; batch adversarial loss: 0.580569
epoch 1; iter: 200; batch classifier loss: 0.538597; batch adversarial loss: 0.527204
epoch 2; iter: 0; batch classifier loss: 0.529805; batch adversarial loss: 0.499234
epoch 2; iter: 200; batch classifier loss: 0.417079; batch adversarial loss: 0.425837
epoch 3; iter: 0; batch classifier loss: 0.394124; batch adversarial loss: 0.486551
epoch 3; iter: 200; batch classifier loss: 0.359398; batch adversarial loss: 0.429863
epoch 4; iter: 0; batch classifier loss: 0.398436; batch adversarial loss: 0.446331
epoch 4; iter: 200; batch classifier loss: 0.476476; batch adversarial loss: 0.404803
epoch 5; iter: 0; batch classifier loss: 0.432306; batch adversarial loss: 0.375442
epoch 5; iter: 200; batch 

### Independencia (Demographic Parity)

In [37]:
get_demographic_parity_metrics(
    df_test_aif,
    predicted_test_inprocessing_1
)

Disparate Impact for age: 1.0136371951791556
Disparate Impact for sex: 0.9397440669775219


(<aif360.metrics.classification_metric.ClassificationMetric at 0x1655bfecbf0>,
 <aif360.metrics.classification_metric.ClassificationMetric at 0x1655bc97ef0>)

### Separación (Equalized Odds)

In [38]:
get_equalized_odds_metrics(
    X_test_df,
    y_test,
    predicted_test_inprocessing_1.labels
)

Age Group: Older Adults
  True Positive Rate (TPR): 0.47
  False Positive Rate (FPR): 0.07
Age Group: Young Adults
  True Positive Rate (TPR): 0.51
  False Positive Rate (FPR): 0.07
Sex Group: Female
  True Positive Rate (TPR): 0.74
  False Positive Rate (FPR): 0.09
Sex Group: Male
  True Positive Rate (TPR): 0.46
  False Positive Rate (FPR): 0.05


### Suficiencia (Predictive Parity)

In [39]:
get_predictive_parity_metrics(
    df_test_aif,
    predicted_test_inprocessing_1
)

Privileged PPV age: 0.7095782073813708
Unprivileged PPV age: 0.7208121827411168
Predictive Parity Difference age: -0.011233975359746018
Privileged PPV sex: 0.7310683585755219
Unprivileged PPV sex: 0.7130177514792899
Predictive Parity Difference sex: 0.018050607096231963


### Accuracy

In [40]:
accuracy_score(y_test, predicted_test_inprocessing_1.labels)

0.8305452509274165

## Combinación 2: In-procesamiento + Post-Procesamiento

In [41]:
## Código que combine las dos técnicas
predicted_test_inprocessing_2 = adversarialDebiasingProcessing(
    train_aif_df=df_train_aif,
    test_aif_df=df_test_aif,
    sensitive_features=['age', 'sex'],
    name_run_identifier='test_3'
)

post_processed_preds_2 = eqOddsPredictionProccesing(
    test_aif_df=df_test_aif,
    test_pred_aif_df=predicted_test_inprocessing_2,
    sensitive_features=['age', 'sex'],
)

Entrenando modelo debiasado para age...
epoch 0; iter: 0; batch classifier loss: 0.676101; batch adversarial loss: 0.731749
epoch 0; iter: 200; batch classifier loss: 0.402378; batch adversarial loss: 0.622374
epoch 1; iter: 0; batch classifier loss: 0.353756; batch adversarial loss: 0.594891
epoch 1; iter: 200; batch classifier loss: 0.426250; batch adversarial loss: 0.519466
epoch 2; iter: 0; batch classifier loss: 0.421696; batch adversarial loss: 0.518851
epoch 2; iter: 200; batch classifier loss: 0.454373; batch adversarial loss: 0.471482
epoch 3; iter: 0; batch classifier loss: 0.489997; batch adversarial loss: 0.454455
epoch 3; iter: 200; batch classifier loss: 0.425716; batch adversarial loss: 0.446720
epoch 4; iter: 0; batch classifier loss: 0.494585; batch adversarial loss: 0.391426
epoch 4; iter: 200; batch classifier loss: 0.510633; batch adversarial loss: 0.326643
epoch 5; iter: 0; batch classifier loss: 0.449365; batch adversarial loss: 0.388905
epoch 5; iter: 200; batch 

### Independencia (Demographic Parity)

In [42]:
# Código de Independencia
get_demographic_parity_metrics(
    df_test_aif,
    post_processed_preds_2
)

Disparate Impact for age: 1.1415396764504229
Disparate Impact for sex: 0.5567263263528193


(<aif360.metrics.classification_metric.ClassificationMetric at 0x1655d1a5be0>,
 <aif360.metrics.classification_metric.ClassificationMetric at 0x1655f09fbf0>)

### Separación (Equalized Odds)

In [43]:
# Código de Separación
get_equalized_odds_metrics(
    X_test_df,
    y_test,
    post_processed_preds_2.labels
)

Age Group: Older Adults
  True Positive Rate (TPR): 0.43
  False Positive Rate (FPR): 0.05
Age Group: Young Adults
  True Positive Rate (TPR): 0.41
  False Positive Rate (FPR): 0.05
Sex Group: Female
  True Positive Rate (TPR): 0.39
  False Positive Rate (FPR): 0.05
Sex Group: Male
  True Positive Rate (TPR): 0.41
  False Positive Rate (FPR): 0.05


### Suficiencia (Predictive Parity)

In [44]:
# Código de Suficiencia
get_predictive_parity_metrics(
    df_test_aif,
    post_processed_preds_2
)

Privileged PPV age: 0.7356651376146789
Unprivileged PPV age: 0.7588235294117647
Predictive Parity Difference age: -0.023158391797085787
Privileged PPV sex: 0.7310683585755219
Unprivileged PPV sex: 0.7130177514792899
Predictive Parity Difference sex: 0.018050607096231963


### Accuracy

In [45]:
accuracy_score(y_test, post_processed_preds_2.labels)

0.8213760761531462

## Combinación 3: Pre-procesamiento + Post-Procesamiento

In [46]:
## Código que combine las dos técnicas
reweighted_train = reweighingPreprocessing(
    train_aif_df=df_train_aif,
    sensitive_features=['age', 'sex'],
)

logistic_model_pre = LogisticRegression(max_iter=5000, random_state=42)
logistic_model_pre.fit(
    reweighted_train.features, 
    reweighted_train.labels.ravel(), 
    sample_weight=reweighted_train.instance_weights
)
print(logistic_model_pre)
y_pred_logistic_pre = logistic_model_pre.predict(X_test_df)



post_processed_preds_3 = eqOddsPredictionProccesing(
    df_test_aif,
    BinaryLabelDataset(
        df=pd.concat([X_test_df, pd.DataFrame(y_pred_logistic_pre, columns=['income'])], axis=1),
        label_names=['income'],
        protected_attribute_names=['age', 'sex'],
    ),
    ['age', 'sex'],
)

LogisticRegression(max_iter=5000, random_state=42)


c:\Users\lucas\python-envs\nb-env\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### Independencia (Demographic Parity)

In [47]:
# Código de Independencia
get_demographic_parity_metrics(
    df_test_aif,
    post_processed_preds_3
)

Disparate Impact for age: 1.0934686605777317
Disparate Impact for sex: 0.5934316437756215


(<aif360.metrics.classification_metric.ClassificationMetric at 0x1655ef02540>,
 <aif360.metrics.classification_metric.ClassificationMetric at 0x1655bef9c10>)

### Separación (Equalized Odds)

In [48]:
# Código de Separación
get_equalized_odds_metrics(
    X_test_df,
    y_test,
    post_processed_preds_3.labels
)

Age Group: Older Adults
  True Positive Rate (TPR): 0.53
  False Positive Rate (FPR): 0.08
Age Group: Young Adults
  True Positive Rate (TPR): 0.52
  False Positive Rate (FPR): 0.07
Sex Group: Female
  True Positive Rate (TPR): 0.51
  False Positive Rate (FPR): 0.08
Sex Group: Male
  True Positive Rate (TPR): 0.53
  False Positive Rate (FPR): 0.07


### Suficiencia (Predictive Parity)

In [49]:
# Código de Suficiencia
get_predictive_parity_metrics(
    df_test_aif,
    post_processed_preds_3
)

Privileged PPV age: 0.6902876198416007
Unprivileged PPV age: 0.7142857142857143
Predictive Parity Difference age: -0.023998094444113605
Privileged PPV sex: 0.7310683585755219
Unprivileged PPV sex: 0.7130177514792899
Predictive Parity Difference sex: 0.018050607096231963


### Accuracy

In [50]:
accuracy_score(y_test, post_processed_preds_3.labels)

0.8283054525092741

## Combinación 4: Pre-procesamiento + In-Procesamiento + Post-Procesamiento

In [51]:
## Código que combine las dos técnicas
rewighted_train = reweighingPreprocessing(
    train_aif_df=df_train_aif,
    sensitive_features=['age', 'sex'],
)

predicted_test_inprocessing_4 = adversarialDebiasingProcessing(
    train_aif_df=rewighted_train,
    test_aif_df=df_test_aif,
    sensitive_features=['age', 'sex'],
    name_run_identifier='test_4'
)

post_processed_preds_4 = eqOddsPredictionProccesing(
    df_test_aif,
    predicted_test_inprocessing_4,
    ['age', 'sex'],
)

Entrenando modelo debiasado para age...
epoch 0; iter: 0; batch classifier loss: 0.777549; batch adversarial loss: 1.184572
epoch 0; iter: 200; batch classifier loss: 1.427868; batch adversarial loss: 1.104806
epoch 1; iter: 0; batch classifier loss: 1.680545; batch adversarial loss: 1.040749
epoch 1; iter: 200; batch classifier loss: 1.638614; batch adversarial loss: 0.769586
epoch 2; iter: 0; batch classifier loss: 1.473621; batch adversarial loss: 0.686931
epoch 2; iter: 200; batch classifier loss: 0.839579; batch adversarial loss: 0.544076
epoch 3; iter: 0; batch classifier loss: 0.648701; batch adversarial loss: 0.494866
epoch 3; iter: 200; batch classifier loss: 0.424854; batch adversarial loss: 0.452438
epoch 4; iter: 0; batch classifier loss: 0.321262; batch adversarial loss: 0.442021
epoch 4; iter: 200; batch classifier loss: 0.282002; batch adversarial loss: 0.432926
epoch 5; iter: 0; batch classifier loss: 0.398796; batch adversarial loss: 0.389925
epoch 5; iter: 200; batch 

### Independencia (Demographic Parity)

In [52]:
# Código de Independencia
get_demographic_parity_metrics(
    df_test_aif,
    post_processed_preds_4
)

Disparate Impact for age: 1.0866878267676674
Disparate Impact for sex: 0.5983967732095041


(<aif360.metrics.classification_metric.ClassificationMetric at 0x1655f2dec30>,
 <aif360.metrics.classification_metric.ClassificationMetric at 0x1653aec8980>)

### Separación (Equalized Odds)

In [53]:
# Código de Separación
get_equalized_odds_metrics(
    X_test_df,
    y_test,
    post_processed_preds_4.labels
)

Age Group: Older Adults
  True Positive Rate (TPR): 0.47
  False Positive Rate (FPR): 0.07
Age Group: Young Adults
  True Positive Rate (TPR): 0.46
  False Positive Rate (FPR): 0.07
Sex Group: Female
  True Positive Rate (TPR): 0.48
  False Positive Rate (FPR): 0.07
Sex Group: Male
  True Positive Rate (TPR): 0.46
  False Positive Rate (FPR): 0.07


### Suficiencia (Predictive Parity)

In [54]:
# Código de Suficiencia
get_predictive_parity_metrics(
    df_test_aif,
    post_processed_preds_4
)

Privileged PPV age: 0.688177107866227
Unprivileged PPV age: 0.7208121827411168
Predictive Parity Difference age: -0.03263507487488981
Privileged PPV sex: 0.7310683585755219
Unprivileged PPV sex: 0.7130177514792899
Predictive Parity Difference sex: 0.018050607096231963


### Accuracy

In [55]:
accuracy_score(y_test, post_processed_preds_4.labels)

0.8196962273395394